In [10]:
import os

In [11]:
os.chdir("../")

In [12]:
%pwd

'/Users/mymac/Documents'

In [13]:
os.chdir("wine-quality")

In [14]:
%pwd

'/Users/mymac/Documents/wine-quality'

In [27]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [28]:
from mlproj.constants import *
from mlproj.utils.common import read_yaml, create_directories

In [29]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        return data_ingestion_config

In [30]:
import os
import zipfile
import urllib.request as request
from mlproj import logger
from mlproj.utils.common import get_size
import requests

In [31]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            response = requests.get(self.config.source_URL)
            
            if 'zip' not in response.headers.get('Content-Type', ''):
                raise ValueError("Downloaded file is not a zip file. Content-Type: " +
                                 response.headers.get('Content-Type', 'unknown'))

            with open(self.config.local_data_file, 'wb') as f:
                f.write(response.content)
            
            logger.info(f"{self.config.local_data_file} downloaded successfully!")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Extracted zip file to: {unzip_path}")
        except zipfile.BadZipFile:
            raise zipfile.BadZipFile(f"The file at {self.config.local_data_file} is not a valid ZIP archive.")

In [32]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-26 10:47:28,548: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-05-26 10:47:28,550: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-26 10:47:28,553: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-26 10:47:28,554: INFO: common: created directory at: artifacts]
[2025-05-26 10:47:28,555: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-26 10:47:28,888: INFO: 1908480698: artifacts/data_ingestion/data.zip downloaded successfully!]
[2025-05-26 10:47:28,894: INFO: 1908480698: Extracted zip file to: artifacts/data_ingestion]
